In [1]:
"""
Ensemble inference – cached model pool & clean conversation template
===================================================================

🔑 **What’s new**
1. **ModelPool (singleton)** caches every HF/vLLM generator and the reward model
   the first time they are requested → later calls reuse them, so repeated
   `run_ensemble()` invocations are *instant*.
2. **ConversationTemplate** – every round is rendered with a fixed, minimal
   format to avoid stray text like “600 words”. History turns are stored in
   `[TURN i]` blocks.
3. Added optional `accumulate_context` flag (default **True**) to control
   whether previously chosen segments are fed back into the next prompt.
4. Still supports unlimited models & EOS‑based early stopping.

Quick usage
-----------
```python
from ensemble_inference import run_ensemble
models = [
    {"path": ".../DeepSeek-R1-Distill-Qwen-1.5B",  "engine": "hf"},
    {"path": ".../DeepSeek-R1-Distill-Qwen-7B",   "engine": "hf"},
    {"path": ".../DeepSeek-R1-Distill-Qwen-14B",  "engine": "vllm"},
]
ans1 = run_ensemble("Explain gradient accumulation.", model_specs=models)
ans2 = run_ensemble("What is RLHF?",                model_specs=models)  # ⬅ reused, no reload
```

Implementation notes
--------------------
* **ModelPool.get_generator() / get_reward()** return already‑loaded instances.
* Conversation prompt:
  ```
  [SYSTEM] You are a helpful assistant. [/SYSTEM]
  [TURN 1]
  <user>
  ...
  </user>
  <assistant>
  ...
  </assistant>
  ...
  [TURN n]
  <user>
  Current question…
  </user>
  <assistant>
  ```
  Last `<assistant>` tag is left open for the model to fill.
* `_clean()` strips trivial length meta like “\d+ words” at line starts.
"""

from __future__ import annotations

import logging
import re
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import torch
import torch.nn.functional as F
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    PreTrainedTokenizerBase,
)

# Optional vLLM backend ------------------------------------------------------
try:
    from vllm import LLM, SamplingParams  # type: ignore

    _VLLM_AVAILABLE = True
except ImportError:  # pragma: no cover
    _VLLM_AVAILABLE = False

# ---------------------------------------------------------------------------
# Logging / constants
# ---------------------------------------------------------------------------
logging.basicConfig(level=logging.INFO, format="[%(levelname)s] %(message)s")
logger = logging.getLogger("ensemble_inference")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EOS_TEXT = ""  # most Qwen / Llama models use empty string as EOS
STEP_TOKEN = "<extra_0>"
SYSTEM_PROMPT = "You are a helpful assistant."
STOP_TOKENS_TEXT = {".", "\n"}  # trimming convenience

# ---------------------------------------------------------------------------
# Conversation template & sanitisation
# ---------------------------------------------------------------------------

def _clean(text: str) -> str:
    """Strip trivial meta lines like "600 words"."""
    return re.sub(r"^\s*\d+\s*words\b.*(?:\n|$)", "", text, flags=re.IGNORECASE | re.MULTILINE).strip()


class ConversationTemplate:
    def __init__(self, system_prompt: str, initial_question: str):
        self.system = system_prompt
        self.turns: List[Tuple[str, str]] = [("user", initial_question)]  # (role, content)

    def add_assistant(self, content: str):
        self.turns[-1] = (self.turns[-1][0], self.turns[-1][1])  # ensure last user ends
        self.turns.append(("assistant", content))

    def new_turn(self, question: str):
        self.turns.append(("user", question))

    def render(self) -> str:
        prompt_lines = [f"[SYSTEM] {self.system} [/SYSTEM]"]
        for idx, (role, content) in enumerate(self.turns, 1):
            if role == "user":
                prompt_lines.append(f"[TURN {idx}]\n<user>\n{content}\n</user>")
            else:  # assistant
                prompt_lines.append(f"<assistant>\n{content}\n</assistant>")
        prompt_lines.append("<assistant>\n")  # leave open for next gen
        return "\n".join(prompt_lines)

# ---------------------------------------------------------------------------
# Helper: trim at first stop token
# ---------------------------------------------------------------------------

def _trim_text(txt: str) -> str:
    for tok in STOP_TOKENS_TEXT:
        pos = txt.find(tok)
        if pos != -1:
            return txt[: pos + len(tok)]
    return txt

# ---------------------------------------------------------------------------
# Step‑probability helper for reward model
# ---------------------------------------------------------------------------

def _step_rewards(logits: torch.Tensor, mask: torch.Tensor):
    probs = F.softmax(logits, dim=-1) * mask.unsqueeze(-1)
    arr: List[List[float]] = []
    for sample in probs:
        pos = sample[sample != 0].view(-1, 2)[:, 1]
        arr.append(pos.cpu().tolist())
    return arr

# ---------------------------------------------------------------------------
# Generator output container
# ---------------------------------------------------------------------------

@dataclass
class GenOutput:
    text: str
    ended_with_eos: bool

# ---------------------------------------------------------------------------
# Generators (HF & vLLM)
# ---------------------------------------------------------------------------

class BaseGenerator:
    name: str

    def generate(self, prompt: str, **kw) -> GenOutput:  # pragma: no cover abstract
        raise NotImplementedError


@dataclass
class HFGenerator(BaseGenerator):
    tokenizer: PreTrainedTokenizerBase
    model: AutoModelForCausalLM

    @classmethod
    def load(cls, path: str, *, dtype: torch.dtype = torch.float16):
        tok = AutoTokenizer.from_pretrained(path, trust_remote_code=True)
        mod = AutoModelForCausalLM.from_pretrained(path, torch_dtype=dtype, device_map="auto", trust_remote_code=True).eval()
        inst = cls(tok, mod)
        inst.name = path
        return inst

    @torch.inference_mode()
    def generate(self, prompt: str, *, max_tokens=128, temperature=0.95, top_p=0.7) -> GenOutput:
        ids = self.tokenizer(prompt, return_tensors="pt").to(DEVICE)
        cfg = GenerationConfig(do_sample=True, temperature=temperature, top_p=top_p, max_new_tokens=max_tokens, pad_token_id=self.tokenizer.eos_token_id)
        out = self.model.generate(**ids, generation_config=cfg)[0]
        ended = bool(out[-1] == self.tokenizer.eos_token_id)
        txt = self.tokenizer.decode(out[len(ids["input_ids"][0]) :], skip_special_tokens=True)
        return GenOutput(_trim_text(_clean(txt)), ended)


class VLLMGenerator(BaseGenerator):
    def __init__(self, path: str):
        if not _VLLM_AVAILABLE:
            raise RuntimeError("vllm not installed")
        self._llm = LLM(model=path)
        self._sp = SamplingParams(max_tokens=128, temperature=0.95, top_p=0.7, stop=list(STOP_TOKENS_TEXT))
        self.name = path
        self._eos_text = EOS_TEXT

    @torch.inference_mode()
    def generate(self, prompt: str, *, max_tokens=128, temperature=0.95, top_p=0.7) -> GenOutput:
        self._sp.max_tokens, self._sp.temperature, self._sp.top_p = max_tokens, temperature, top_p
        txt = self._llm.generate([prompt], self._sp)[0].outputs[0].text
        ended = txt.endswith(self._eos_text)
        return GenOutput(_trim_text(_clean(txt)), ended)

# ---------------------------------------------------------------------------
# ModelPool: caching layer
# ---------------------------------------------------------------------------

class ModelPool:
    _gen_cache: Dict[Tuple[str, str], BaseGenerator] = {}
    _reward_cache: Dict[str, "PRMScorer"] = {}

    @classmethod
    def get_generator(cls, path: str, engine: str = "hf") -> BaseGenerator:
        key = (engine, path)
        if key not in cls._gen_cache:
            logger.info("[Pool] loading %s (%s)", path, engine)
            if engine == "hf":
                cls._gen_cache[key] = HFGenerator.load(path)
            elif engine == "vllm":
                cls._gen_cache[key] = VLLMGenerator(path)
            else:
                raise ValueError(engine)
        return cls._gen_cache[key]

    @classmethod
    def get_reward(cls, path: str) -> "PRMScorer":
        if path not in cls._reward_cache:
            logger.info("[Pool] loading reward model %s", path)
            cls._reward_cache[path] = PRMScorer(path)
        return cls._reward_cache[path]

# ---------------------------------------------------------------------------
# Reward model
# ---------------------------------------------------------------------------

class PRMScorer:
    def __init__(self, path: str):
        self.tok = AutoTokenizer.from_pretrained(path, trust_remote_code=True)
        self.mod = AutoModel.from_pretrained(path, torch_dtype=torch.bfloat16, device_map="auto", trust_remote_code=True).eval()
        self.sep_id = self.tok.encode(STEP_TOKEN)[0]

    @torch.inference_mode()
    def score(self, question: str, answer: str) -> float:
        if not answer.endswith(STEP_TOKEN):
            answer = answer + STEP_TOKEN
        msgs = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer},
        ]
        convo = self.tok.apply_chat_template(msgs, tokenize=False, add_generation_prompt=False)
        ids = self.tok(convo, return_tensors="pt").input_ids.to(DEVICE)
        mask = ids == self.sep_id
        probs = _step_rewards(self.mod(ids).logits, mask)[0]
        return float(sum(probs) / len(probs) * 10.0) if probs else 0.0

# ---------------------------------------------------------------------------
# Ensemble reasoner
# ---------------------------------------------------------------------------

@dataclass
class EnsembleReasoner:
    generators: List[BaseGenerator]
    scorer: PRMScorer
    max_rounds: int = 5
    score_threshold: float = 0.5
    accumulate_context: bool = True

    def __call__(self, question: str) -> str:
        convo = ConversationTemplate(SYSTEM_PROMPT, question)
        collected: List[str] = []

        for rnd in range(1, self.max_rounds + 1):
            prompt = convo.render()
            outs = [g.generate(prompt) for g in self.generators]
            segs = [o.text for o in outs]
            scores = [self.scorer.score(question, s) for s in segs]

            for g, t, s in zip(self.generators, segs, scores):
                logger.info("→ %s | %.2f | %s", g.name, s, t.replace("\n", "\\n"))

            best_idx = int(torch.tensor(scores).argmax())
            best_out = outs[best_idx]
            best_score = scores[best_idx]

            if best_score < self.score_threshold:
                logger.info("Stop: best score %.2f < threshold", best_score)
                break

            collected.append(best_out.text)
            if self.accumulate_context:
                convo.add_assistant(best_out.text)
            if best_out.ended_with_eos:
                logger.info("Early stop: EOS token emitted")
                break

        return " ".join(collected)

# ---------------------------------------------------------------------------
# Public API
# ---------------------------------------------------------------------------

def run_ensemble(
    question: str,
    *,
    model_specs: Optional[List[Dict]] = None,
    max_rounds: int = 5,
    score_threshold: float = 0.5,
    accumulate_context: bool = True,
) -> str:
    if model_specs is None:
        model_specs = [
            {"path": "/root/autodl-tmp/DeepSeek-R1-Distill-Qwen-1.5B", "engine": "hf"},
            {"path": "/root/autodl-tmp/DeepSeek-R1-Distill-Qwen-7B",  "engine": "hf"},
        ]

    gens = [ModelPool.get_generator(spec["path"], spec.get("engine", "hf")) for spec in model_specs]
    scorer = ModelPool.get_reward("/root/autodl-tmp/Qwen2.5-Math-PRM-7B")

    reasoner = EnsembleReasoner(gens, scorer, max_rounds, score_threshold, accumulate_context)
    return reasoner(question)



<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_9943/2633921990.py:1: SyntaxWarning: invalid escape sequence '\d'
  """


INFO 04-29 18:41:55 [__init__.py:239] Automatically detected platform cuda.


In [2]:
# from ensemble_inference import run_ensemble

models = [
    {"path": "/root/autodl-tmp/DeepSeek-R1-Distill-Qwen-1.5B",  "engine": "hf"},
    # {"path": "/root/autodl-tmp/DeepSeek-R1-Distill-Qwen-7B",  "engine": "hf"},
    {"path": "/root/autodl-tmp/Qwen2.5-Math-7B-Instruct",  "engine": "hf"},
]

text = "Convert the point $(0,3)$ in rectangular coordinates to polar coordinates. Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$"

# 第一次加载（会真正载模型）
ans1 = run_ensemble(
    text, 
    model_specs=models,
    max_rounds=100,
    score_threshold=0.1,
)

ans1

<>:8: SyntaxWarning: invalid escape sequence '\l'
<>:8: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_9943/2841911115.py:8: SyntaxWarning: invalid escape sequence '\l'
  text = "Convert the point $(0,3)$ in rectangular coordinates to polar coordinates. Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$"
[INFO] [Pool] loading /root/autodl-tmp/DeepSeek-R1-Distill-Qwen-1.5B (hf)
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
[INFO] We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
[INFO] [Pool] loading /root/autodl-tmp/DeepSeek-R1-Distill-Qwen-7B (hf)
[INFO] We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO] [Pool] loading reward model /root/autodl-tmp/Qwen2.5-Math-PRM-7B
[INFO] We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at /root/autodl-tmp/Qwen2.5-Math-PRM-7B were not used when initializing Qwen2ForProcessRewardModel: ['lm_head.weight']
- This IS expected if you are initializing Qwen2ForProcessRewardModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Qwen2ForProcessRewardModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[WARNING] We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
[INFO] → /root/autodl-tmp/DeepSeek-R1-Distill-Qwen-1.5B | 10.00 | Alright, so I need to conve

"Alright, so I need to convert the point (0,3) from rectangular coordinates to polar coordinates. Okay, let's start by recalling what polar coordinates are. In polar coordinates, a point is represented as (r, θ), where r is the distance from the origin to the point, and θ is the angle made with the positive x-axis. Right, so I have the point (0,3) in rectangular coordinates, which means x is 0 and y is 3. To find r, the formula is r = sqrt(x² + y²). Plugging in the values, x is 0, so 0 squared is 0, and y is 3, so 3 squared is 9. So r = sqrt(0 + 9) = sqrt(9) = 3. Okay, so r is 3. Now, I need to find θ. Since x is 0, the point lies on the y-axis. In rectangular coordinates, if x is 0, the point is either on the positive y-axis or the negative y-axis. Here, y is 3, which is positive, so the point is on the positive y-axis. What angle does the positive y-axis make with the positive x-axis?\n</assistant>\n<assistant>\nI remember that the positive x-axis is 0 radians, and the positive y-axi

In [4]:
# 再次调用（复用缓存，几乎秒回）
ans2 = run_ensemble("但是答案是$\left( 3, \frac{\pi}{2} \right)$", model_specs=models)

ans2

<>:2: SyntaxWarning: invalid escape sequence '\l'
<>:2: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_9943/2405989248.py:2: SyntaxWarning: invalid escape sequence '\l'
  ans2 = run_ensemble("但是答案是$\left( 3, \frac{\pi}{2} \right)$", model_specs=models)
[INFO] → /root/autodl-tmp/DeepSeek-R1-Distill-Qwen-1.5B | 9.57 | 嗯，用户说答案是$(3, \frac{\pi}{2})$，这让我有点困惑。首先，我需要明确这是什么类型的题目。可能涉及到极坐标或者直角坐标系中的转换。\n
[INFO] → /root/autodl-tmp/DeepSeek-R1-Distill-Qwen-7B | 9.88 | Alright, so I have this problem where I need to find the polar coordinates of the point (0, 3).
[INFO] → /root/autodl-tmp/DeepSeek-R1-Distill-Qwen-1.5B | 9.96 | Okay, let's start by recalling how polar coordinates work.
[INFO] → /root/autodl-tmp/DeepSeek-R1-Distill-Qwen-7B | 8.36 | Okay, let me think.
[INFO] → /root/autodl-tmp/DeepSeek-R1-Distill-Qwen-1.5B | 9.80 | In polar coordinates, a point is represented as (r, θ), where r is the distance from the origin, and θ is the angle measured from the positive x-axis.
[INFO] → /

"Alright, so I have this problem where I need to find the polar coordinates of the point (0, 3). Okay, let's start by recalling how polar coordinates work. In polar coordinates, a point is represented as (r, θ), where r is the distance from the origin to the point, and θ is the angle made with the positive x-axis. Given the Cartesian coordinates (x, y), the conversion to polar coordinates involves some trigonometry. Specifically, r is calculated using the formula r = sqrt(x^2 + y^2)."